## 以做圖方式觀察台灣高級中等學校歷年變化

[各級學校位置](https://www.tgos.tw/TGOS/Web/Metadata/TGOS_MetaData_View.aspx?MID=404530389BCC15967ED8D58E129BC680&SHOW_BACK_BUTTON=false)

In [10]:
import pandas as pd

base = pd.DataFrame()
for i in range(103,110):
#    print(i)
    temp = pd.read_csv(f"{i}_base0.csv",thousands=",")
    temp['year'] = f"{i}"
    base = pd.concat([base, temp], ignore_index=True)

base = base.drop(columns=['縣市代碼','縣市名稱'])

#base.info()
#base['year'].unique()

In [11]:
lv_table = pd.read_csv("lv_table.csv",thousands=",")

In [12]:
location = pd.read_excel("110學年度各級學校分布位置_高中職.xlsx")
#location['map_point'] = '(' & location['X 坐標'] & ',' & location['Y 坐標'] & ')'
#location

In [13]:
base['teach_fulltime'] = base['專任教師數男'] + base['專任教師數女']
base['teach_parttime'] = base['兼任教師數男'] + base['兼任教師數女']
base['teach_all'] = base['teach_fulltime'] + base['teach_parttime']
base['emp'] = base['職員數男'] + base['職員數女']
base['stud'] = base['學生數男'] + base['學生數女']
base['grad'] = base['上學年畢業生數男'] + base['上學年畢業生數女']

base['stud/teach_all'] = base['stud'] / base['teach_all']
base['teach_all/stud'] = 1/(base['stud'] / base['teach_all'])
base['stud/grad'] = base['stud'] / base['grad']
base['teach_full/part'] = (base['teach_fulltime'] / base['teach_parttime']).fillna(base['teach_fulltime'])
base['stud/cls'] = base['stud'] / base['班級數']

base['site'] = base['學校代碼'].str[0:2]
base['cate'] = base['學校代碼'].str[2]
base['lv'] = base['學校代碼'].str[3]

#把設立別跟學校級別串接進去，之後比較方便分析
#base109 = base109.merge(cate_table, how='left', left_on='cate', right_on='cate_code')
base = base.merge(lv_table, how='left', left_on='lv', right_on='lv_code')
base = base.merge(location[['代碼','X 坐標','Y 坐標']], how='left', left_on='學校代碼', right_on='代碼')
#print(base109.tail())
#base109.tail()

print(base.tail())
print(base.info())

        學校代碼        學校名稱  專任教師數男  專任教師數女  兼任教師數男  兼任教師數女  職員數男  職員數女  班級數  \
3661  151G91  私立正德高中進修學校       0       0      23       5     0     0    3   
3662  03C301    誠正中學桃園分校       0       0       0       0     0     0   11   
3663  04C301        誠正中學       0       0       0       0     0     0   13   
3664  07C301    誠正中學彰化分校       0       0       0       0     0     0   14   
3665  12C301        明陽中學       0       0       0       0     0     0    6   

      學生數男  ...  teach_full/part   stud/cls  site cate  lv  lv_code  lv_name  \
3661    71  ...              0.0  23.666667    15    1   G        G     監獄補校   
3662   177  ...              0.0  16.090909    03    C   3        3     高級中學   
3663   210  ...              0.0  16.153846    04    C   3        3     高級中學   
3664   148  ...              0.0  14.714286    07    C   3        3     高級中學   
3665    61  ...              0.0  10.333333    12    C   3        3     高級中學   

       代碼  X 坐標  Y 坐標  
3661  NaN   NaN   NaN  
3662  Na

In [5]:
import plotly.express as px

#base_01 = base.dropna(subset=['stud/teach_all'])
#px.scatter(base.query("lv_name == '高級中學' or lv_name == '高級職校'")
#           , x="stud", y="teach_all"
#           , color="lv_name", size="stud/teach_all", hover_name="學校名稱", animation_frame = "year")

### Q1：哪些學校師生比最高（一個老師要顧最多學生）？
### Q2：哪些學校平均班級人數最少？
### Q3：從103年到109年，有什麼變化？

In [14]:
fig = px.scatter(base.query("lv_name == '高級中學' or lv_name == '高級職校'")
                 , x="stud/teach_all", y="stud/cls"
                 , color="lv_name", size="stud", hover_name="學校名稱", animation_frame = "year")
#fig.update_xaxes(range=[0, 0.15])
fig.show()

### Q1：哪些學校師生比最高（一個老師要顧最多學生）？
私立巨人高中一直都有很高的師生比，109年是36學生/老師，嘉揚高中(39.57)及新光高中(44.14)在109年也有很高的師生比。
以高職來說，永平工商、開平餐飲、萬能工商、光隆家商等，都曾經在歷年資料中有最高的師生比，但較令人注目的仍是中山工商，學生數量最多(109年有8449人)，師生比維持在20以上，平均每一班的人數也保持40人以上。
### Q2：哪些學校平均班級人數最少？班級人數跟師生比是不是正相關(班級人數越少，師生比越低)？
私立漢英高中(4.5)及南海月光實驗中學(9.3)有最少的平均班級人數，私立芳曙商工(7.6)是高職中最低者。
高職有較明顯的負相關情況，班級人數一多，師生比就變大，高中則在不同的師生比狀態下，都有不同的班級人數。
### Q3：從103年到109年，有什麼變化？
1. 高職師生比減少、班級學生數變少：可從高職資料點的持續往左下移動看出趨勢
2. 高中各校學生數量有減少趨勢：可以從藍色資料點的縮小看出
3. 高中的師生比及平均班級人數有分散趨勢，可能代表不同高中嘗試不同教學特色

In [7]:
fig = px.scatter(base
                 , x='X 坐標', y='Y 坐標'
                 , color="lv_name", size="stud", hover_name="學校名稱", animation_frame = "year")
#fig.update_xaxes(range=[0, 0.15])
fig.show()

In [8]:
#嘗試使用地圖做圖，尚未成功
base_map = base.dropna()
#base_map.info()
fig = px.scatter_geo(base_map, lon = 'X 坐標', lat = 'Y 坐標', color="lv_name"
                     ,hover_name="學校名稱", size="stud"
                     ,projection="natural earth", scope="asia")
fig.show()